In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# 第一步 构建数据管道并导入图片数据集 (分割为训练集与测试集 5：5)

In [2]:
# data_dir 是你存放data数据的地址，我这里放在了相对路径下
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# 构建数据预处理
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# 导入数据到 dataset 中
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# 现在将数据集分为训练集与测试集
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# 最后构建数据管道
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# 第二步 构建模型 定制损失函数与优化器

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d 参数 nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d 参数 nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3,  32, 3, 1, padding = "same"),                              
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # 全连接层，输出 21 个类别
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # 扁平化处理 x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# 选择是否打印模型，这里就不打印了，直接注释
# print(model)

# 选择损失函数 以及 优化器
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

# 第三步 传入模型开始训练与测试

In [6]:
# 训练模型 并记录训练时 每个epoch 的 准确率与损失值
def train(num_epochs, model, loaders):
    model.train()   
    
    # 第一个存储正确率，第二个存储损失值
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # 清理之前的梯度 
            optimizer.zero_grad()           
            
            # 进行反向传播 然后更新参数
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # 将该epoch 的 正确率与损失值 添加到相应列表
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.0277
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 3.0651
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 2.9583
epoch: 01 | Accuracy: 5.11 Loss: 82.0500

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 3.0630
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.7866
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.5439
epoch: 02 | Accuracy: 12.49 Loss: 77.7245

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 2.8609
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 2.3130
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 2.2799
epoch: 03 | Accuracy: 24.02 Loss: 67.9408

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 2.0901
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 2.0328
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 2.2339
epoch: 04 | Accuracy: 34.13 Loss: 59.0928

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 1.3852
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 1.2242
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 1.2541
epoch: 05 | Accuracy: 46.14 Loss: 47.0855

Epoch: [6/30]	|	Step: [9/27]	|	Loss: 1.6450
Epoch: [6/30]	|	Step: [18/27]	|	Loss: 1.1708
Epoch: [6/30]	|	Step: [27/2

In [7]:
print(Accy_list)

[5.1129608154296875, 12.485136985778809, 24.019025802612305, 34.126041412353516, 46.13555145263672, 59.453033447265625, 70.74910736083984, 84.30439758300781, 85.25564575195312, 90.48751831054688, 94.53031921386719, 96.55172729492188, 96.19500732421875, 97.38406372070312, 99.40547180175781, 98.57312774658203, 99.04875183105469, 99.16765594482422, 99.16765594482422, 99.88109588623047, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [8]:
print(Loss_list)

[82.04997205734253, 77.72445964813232, 67.94083499908447, 59.092793464660645, 47.085500717163086, 35.42876297235489, 25.82498151063919, 15.769253253936768, 13.227836430072784, 8.232610315084457, 5.304836608469486, 3.1999749839305878, 3.4779706057161093, 2.6435604225844145, 0.9553048382513225, 1.3906892710365355, 1.2841642568819225, 0.9770859987474978, 0.8196375574916601, 0.29862227267585695, 0.13722559821326286, 0.07959511369699612, 0.055616957833990455, 0.04717749271367211, 0.04083080741111189, 0.03568980390264187, 0.032209269382292405, 0.028388361097313464, 0.026825048436876386, 0.02352506827446632]


In [9]:
# 测试模型
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # 获取损失值 并进行累加
            loss = loss_func(outputs, labels)
            L += loss.item()
            
            # 获取正确率
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 72.29 %, Loss: 65.5122 
